In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

# change to qt5 or other backend if tkinter is not installed
%matplotlib tk

### Usage
- Change the `PATH` variable to the path of the folder containing the images.
- Change the `IMG_FORMAT` variable to the format of the images. (e.g. `jpg`, `png`, etc.)
- Change the `POINTS` variable to the number of points you want to make.

### Controls
Use **RIGHT CLICK** to make a point.

### Ouput
Creates a numpy array on shape `(POINTS x NUMBER_OF_IMAGES x 2)` in the same folder as the images.

In [ ]:
PATH       : Path = Path("PATH TO YOUR DATA")     # directory of the images
IMG_FORMAT : str  = "EXTENSION OF YOUR DATA"      # e.g. jpg/jpeg/png...
POINTS     : int  = NUMBER_OF_POINTS              # number of points 

In [2]:
# # sample values
PATH = Path.cwd()/'demo'
IMG_FORMAT = "jpg"
POINTS = 3

In [3]:
# safeguard to prevent overwriting of previously generated points
if (PATH/"points.npy").exists():
    raise Exception("points.npy already exists. Delete the points.npy file to start over.")

Exception: points.npy already exists. Delete the points.npy file to start over.

In [4]:
IMAGES = sorted([x for x in PATH.iterdir() if x.suffix == f".{IMG_FORMAT}"])

colors = ['red', 'green', 'blue', 'yellow',
          'orange', 'purple', 'pink', 'brown']
fig, ax = plt.subplots(nrows=1, ncols=len(IMAGES))
for i, img in enumerate(IMAGES):
    ax[i].imshow(plt.imread(img.absolute()))

ptind    = 0
imgind   = 0
# (POINTS X NUMBER OF IMAGES X 2) array
COORDS = np.zeros((POINTS, len(IMAGES), 2), dtype=np.int64)

def onclick(event):
    
    global ptind
    global imgind

    if event.xdata == None or event.ydata == None: return # click outside of image
    if event.button != 3: return                          # ensure right click
    if event.inaxes != ax[imgind]: return                 # ensure click on the correct image

    ix, iy = event.xdata, event.ydata
    COORDS[ptind][imgind][0] = round(ix)
    COORDS[ptind][imgind][1] = round(iy)

    color = colors[ptind % len(colors)]
    ax[imgind].scatter(ix, iy, color=color, marker='x', s=10)
    ax[imgind].text(ix, iy, f"{ptind+1}", color=color, fontsize=10)
    
    if imgind == len(IMAGES) - 1: ptind += 1
    imgind = (imgind + 1) % len(IMAGES)
    
    plt.draw()
    plt.pause(0.01)
    
    if ptind == POINTS:
        points_path = PATH/"points.npy"
        np.save(str(points_path), COORDS)
        print(f"Saved points to {points_path.absolute()}. You can close the window now.")
        fig.canvas.mpl_disconnect(cid)

cid = fig.canvas.mpl_connect('button_press_event', onclick)